In [10]:
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default="plotly_white"

metro_data=pd.read_csv("Delhi-Metro-Network.csv")

In [12]:
print(metro_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Station ID                285 non-null    int64  
 1   Station Name              285 non-null    object 
 2   Distance from Start (km)  285 non-null    float64
 3   Line                      285 non-null    object 
 4   Opening Date              285 non-null    object 
 5   Station Layout            285 non-null    object 
 6   Latitude                  285 non-null    float64
 7   Longitude                 285 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 17.9+ KB
None


In [17]:
print(metro_data)

     Station ID           Station Name  Distance from Start (km)  \
0             1               Jhil Mil                      10.3   
1             2    Welcome [Conn: Red]                      46.8   
2             3            DLF Phase 3                      10.0   
3             4             Okhla NSIC                      23.8   
4             5             Dwarka Mor                      10.2   
..          ...                    ...                       ...   
280         281             Bata Chowk                      38.3   
281         282       Dwarka Sector 12                       5.8   
282         283        Noida Sector 18                      43.6   
283         284      Knowledge Park II                      21.4   
284         285  Mayur Vihar Extention                      39.5   

             Line Opening Date Station Layout   Latitude  Longitude  
0        Red line   2008-04-06       Elevated  28.675790  77.312390  
1       Pink line   2018-10-31       Elevat

In [18]:
print(len(metro_data))

285


In [14]:
missing_values=metro_data.isnull().sum()

print(missing_values)

Station ID                  0
Station Name                0
Distance from Start (km)    0
Line                        0
Opening Date                0
Station Layout              0
Latitude                    0
Longitude                   0
dtype: int64


In [15]:
metro_data['Opening Date']=pd.to_datetime(metro_data['Opening Date'])

In [21]:
print(metro_data['Line'])

0          Red line
1         Pink line
2       Rapid Metro
3      Magenta line
4         Blue line
           ...     
280     Voilet line
281       Blue line
282       Blue line
283       Aqua line
284       Blue line
Name: Line, Length: 285, dtype: object


In [23]:
line_colors={
    'Red line':'red',
     'Blue line': 'blue',
    'Yellow line': 'beige',
    'Green line': 'green',
    'Voilet line': 'purple',
    'Pink line': 'pink',
    'Magenta line': 'darkred',
    'Orange line': 'orange',
    'Rapid Metro': 'cadetblue',
    'Aqua line': 'black',
    'Green line branch': 'lightgreen',
    'Blue line branch': 'lightblue',
    'Gray line': 'lightgray'
}

delhi_map_with_line_tooltip=folium.Map(location=[28.7041,77.1025],zoom_start=11)

for index,row in metro_data.iterrows():
    line=row['Line']
    color=line_colors.get(line,'black')
    folium.Marker(
         location=[row['Latitude'],row['Longitude']],
         popup=f"{row['Station Name']}",
        tooltip=f"{row['Station Name']},{line}",
        icon=folium.Icon(color=color)
    ).add_to(delhi_map_with_line_tooltip)
    


delhi_map_with_line_tooltip

In [31]:
metro_data['Opening Year']=metro_data['Opening Date'].dt.year

stations_per_year=metro_data['Opening Year'].value_counts().sort_index()

stations_per_year_df=stations_per_year.reset_index()
stations_per_year_df.columns=['Year', 'Number of Stations']

fig=px.bar(stations_per_year_df,x='Year',y='Number of Stations',title="Number of Metro Stations Opened Each Year in Delhi",
 labels={'Year': 'Year', 'Number of Stations': 'Number of Stations Opened'})

fig.update_layout(xaxis_tickangle=-45, xaxis=dict(tickmode='linear'),
                  yaxis=dict(title='Number of Stations Opened'),
                  xaxis_title="Year")



fig.show()

In [45]:
stations_per_line=metro_data['Line'].value_counts()

total_max_distance=metro_data.groupby('Line')['Distance from Start (km)'].max()

avg_distance_per_line=total_max_distance/(stations_per_line.values-1)

line_analysis=pd.DataFrame({
    'Line':stations_per_line.index,
    'Number of Stations':stations_per_line.values,
     'Average Distance Between Stations (km)': avg_distance_per_line
})


line_analysis=line_analysis.sort_values(by='Number of Stations',ascending=False)

line_analysis.reset_index(drop=True,inplace=True)

line_analysis



,Line,Number of Stations,Average Distance Between Stations (km)
0,Blue line,49,0.564583
1,Pink line,38,1.424324
2,Yellow line,37,0.225000
3,Voilet line,34,0.118182
4,Red line,29,0.885714
5,Magenta line,25,0.087500
6,Aqua line,21,1.655000
7,Green line,21,1.040000
8,Rapid Metro,11,5.260000
9,Blue line branch,8,1.428571


In [48]:
# creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Number of Stations Per Metro Line',
                                                    'Average Distance Between Stations Per Metro Line'),
                    horizontal_spacing=0.2)

# plot for Number of Stations per Line
fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Number of Stations'],
           orientation='h', name='Number of Stations', marker_color='crimson'),
    row=1, col=1
)

# plot for Average Distance Between Stations
fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Average Distance Between Stations (km)'],
           orientation='h', name='Average Distance (km)', marker_color='navy'),
    row=1, col=2
)

# update xaxis properties
fig.update_xaxes(title_text="Number of Stations", row=1, col=1)
fig.update_xaxes(title_text="Average Distance Between Stations (km)", row=1, col=2)

# update yaxis properties
fig.update_yaxes(title_text="Metro Line", row=1, col=1)
fig.update_yaxes(title_text="", row=1, col=2)

# update layout
fig.update_layout(height=600, width=1200, title_text="Metro Line Analysis", template="plotly_white")

fig.show()